In [4]:
#To provide server to VPN_access via PC
import os
os.environ['http_proxy'] = 'http://10.162.68.203:5555'
os.environ['https_proxy'] = 'http://10.162.68.203:5555'

In [5]:
import torch
import torchvision.models as models
import torchprofile
import time
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
from torchsummary import summary

In [6]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from torchinfo import summary
import torch
import torch.nn as nn
from transformers import MobileNetV1ForImageClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = MobileNetV1ForImageClassification.from_pretrained("google/mobilenet_v1_1.0_224")

# Modify classifier layer
model.classifier = nn.Linear(in_features=1024, out_features=2)
model.to(device)

# Run model summary using torchinfo
summary(model, input_size=(1, 3, 256, 256), col_names=["input_size", "output_size", "num_params"])


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #
MobileNetV1ForImageClassification             [1, 3, 256, 256]          [1, 2]                    --
├─MobileNetV1Model: 1-1                       [1, 3, 256, 256]          [1, 1024]                 --
│    └─MobileNetV1ConvLayer: 2-1              [1, 3, 256, 256]          [1, 32, 128, 128]         --
│    │    └─Conv2d: 3-1                       [1, 3, 257, 257]          [1, 32, 128, 128]         864
│    │    └─BatchNorm2d: 3-2                  [1, 32, 128, 128]         [1, 32, 128, 128]         64
│    │    └─ReLU6: 3-3                        [1, 32, 128, 128]         [1, 32, 128, 128]         --
│    └─ModuleList: 2-2                        --                        --                        --
│    │    └─MobileNetV1ConvLayer: 3-4         [1, 32, 128, 128]         [1, 32, 128, 128]         352
│    │    └─MobileNetV1ConvLayer: 3-5         [1, 32, 128, 128]         [1, 64, 128,

In [5]:
input_tensor = torch.randn(1, 3, 256, 256).to(device)

https://github.com/zhijian-liu/torchprofile

In [6]:
# Warm up the GPU
with torch.no_grad():
    for _ in range(10):
        _ = model(input_tensor)

# Measure FLOPs
with torch.no_grad():
    macs = torchprofile.profile_macs(model, args=(input_tensor,))
    flops = 2 * macs  # Convert MACs to FLOPs

# Measure inference time
with torch.no_grad():
    start_time = time.time()
    for _ in range(100):  # Run multiple inferences
        _ = model(input_tensor)
    end_time = time.time()

# Calculate elapsed time per inference and throughput
elapsed_time = (end_time - start_time) / 100  # Average inference time per run
tops = (flops / elapsed_time) / 1e12  # Throughput in TOPs

/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::silu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::im2col". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::split_with_sizes". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::expand_as". Skipped.
  warnin

In [7]:
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs, "
        f"Inference Time: {elapsed_time * 1000:.2f} ms, Throughput: {tops:.2f} TOPs")

FLOPs: 3.65 GFLOPs, Inference Time: 10.66 ms, Throughput: 0.34 TOPs
